In [2]:
import csv
import xml.etree.ElementTree as ET
import requests
import pandas as pd
import os
import concurrent.futures
from pandas.errors import EmptyDataError

In [3]:
# Getting country names and iso codes

overwrite = False # Manually change if file overwrite needed
iso_path = "./data/trade_country_iso.csv"

# Check if API request already done
if not(os.path.exists(iso_path)) or overwrite:

    url = "http://wits.worldbank.org/API/V1/wits/datasource/tradestats-trade/country/all"

    response = requests.get(url)

    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)

        # Find the countries element
        countries_element = root.find("{http://wits.worldbank.org}countries")

        if countries_element is not None:
            # Find all country elements
            countries = countries_element.findall("{http://wits.worldbank.org}country")

            # Save country names as csv
            iso_file = open(iso_path,'w', newline='')
            writer = csv.writer(iso_file)
            writer.writerow(['state_name','iso3'])  # Write header
            for country in countries:
                writer.writerow([country.find("{http://wits.worldbank.org}name").text,country.find("{http://wits.worldbank.org}iso3Code").text])
            iso_file.close()

            print(f"Country names saved to {iso_path}.")

        else:
            print("No countries found in the response.")
    else:
        print("Failed to retrieve data. Status code:", response.status_code)

iso_df = pd.read_csv(iso_path,encoding='latin1')
iso_df.head()

,state_name,iso3
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [4]:
# Get all indicators and descriptions

overwrite = False # Manually change if file overwrite needed
indicator_path = "./data/indicator_data.xml"
# Check if API request already done
if not(os.path.exists(indicator_path)) or overwrite:
    url = "http://wits.worldbank.org/API/V1/wits/datasource/tradestats-trade/indicator/ALL"

    # Make a GET request to the API endpoint
    response = requests.get(url)


    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)

        # Find the countries element
        indicators_element = root.find("{http://wits.worldbank.org}indicators")

        if indicators_element is not None:
            indicator_file = open(indicator_path,'wb')
            indicator_file.write(response.content)
            indicator_file.close()
            print(f"Indicators save to {indicator_file}.")
        else:
            print("Error: root element not found")
    else:
        # Print an error message if the request was not successful
        print("Error:", response.status_code)

# Loading in xml file
tree = ET.parse(indicator_path)
root = tree.getroot()

# Initialise lists to store data
data = []
indicators = root.find("{http://wits.worldbank.org}indicators")

# Iterate over each indicator element
for indicator in indicators:
    # Some needed info is stored as an attribute
    indicator_data = indicator.attrib
    # Iterate through indicator to get additional fields
    for child in indicator:
        indicator_data[child.tag.removeprefix("{http://wits.worldbank.org}")] = child.text
    data.append(indicator_data)

# Dataframe with some columns dropped and reorganised, so I can actually read this
indicators_df = pd.DataFrame(data).drop(["topic","source","productclassification","currency","periodicity"],axis=1).reindex(columns=['name','indicatorcode','definition','ispartnerequired','SDMX_partnervalue','isproductrequired','SDMX_productvalue','valuation','notes'])

"""Important indicator codes to note: 'MPRT-TRD-VL' & 'XPRT-TRD-VL' as import and export
trade value measured in US$ Thousands and 'NMBR-MPRT-PRTNR' & 'NMBR-XPRT-PRTNR' as number of import and export partners"""
indicators_df

,name,indicatorcode,definition,ispartnerequired,SDMX_partnervalue,isproductrequired,SDMX_productvalue,valuation,notes
0,Country Growth (%),CNTRY-GRWTH,Annual percentage growth rate of the country’s...,yes,,yes,,Export - FOB; Import - CIF,1) Mirror Exports is considered 2)Growth for a...
1,HH Market concentration index,HH-MKT-CNCNTRTN-NDX,Hirschman Herfindahl index is a measure of th...,no,999,no,999999,Export - FOB; Import - CIF,1) Mirror Exports is considered for export dat...
2,Index Of Export Market Penetration,NDX-XPRT-MKT-PNRTTN,It is calculated as the number of countries t...,no,999,no,999999,Export - FOB; Import - CIF,1) Mirror Exports is considered for export dat...
3,No Of traded HS6 digit Products,NMBR-MPRT-HS6-PRDCT,Total number of products imported by a country...,yes,,no,999999,None,1) All traded products at HS 6 digits are con...
4,No Of traded HS6 digit Products,NMBR-XPRT-HS6-PRDCT,Total number of products exported by a country...,yes,,no,999999,None,1) All traded products at HS 6 digits are con...
5,Number of export partners,NMBR-XPRT-PRTNR,Number of countries to whom a particular count...,no,999,no,999999,None,1) Product 'Total' is used for this computation.
6,Number of import partners,NMBR-MPRT-PRTNR,Number of countries from which a particular co...,no,999,no,999999,None,1) Product 'Total' is used for this computation.
7,Number of products exported,NMBR-PRDCT-XPRTD,Total number of products exported by a country...,no,999,no,999999,None,1) All traded products at HS 6 digits are con...
8,Number of products imported,NMBR-PRDCT-MPRTD,Total number of products imported by a country...,no,999,no,999999,None,1) All traded products at HS 6 digits are con...
9,Partner Share (%),XPRT-PRTNR-SHR,The share of total merchandise trade (export o...,yes,,no,999999,None,None


In [5]:
# Finds the common iso codes between the alliances and trade data

# Get all countries and their iso codes and corresponding CCodes
alliance_codes = pd.read_csv(os.path.join(os.getcwd(),"..","COW/COW_data/COW-country-codes.csv"),encoding='latin1').drop_duplicates()
trade_iso_path = "./data/trade_country_iso.csv"
trade_iso_codes = pd.read_csv(trade_iso_path,encoding='latin1')

# Find the intersection of the two dataframes
merged_inner_df = pd.merge(trade_iso_codes, alliance_codes, left_on='iso3', right_on='StateAbb', how='inner')

# Filter the merged DataFrame to get entries where 'iso3' matches 'StateAbb'
matched_entries = merged_inner_df[['iso3', 'StateAbb', 'state_name','CCode']]

# Find the union of the two dataframes
unmatched_alliance_entries = alliance_codes

# Find all the unmatched entries
for i in unmatched_alliance_entries.iterrows():
    ccode = i[1]['CCode']
    if ccode in alliance_codes['CCode'].values:
        # Remove the entry from unmatched_entries
        unmatched_alliance_entries = unmatched_alliance_entries[~(unmatched_alliance_entries['CCode'] == ccode)]

# Create a dataframe to store the unmatched iso codes
unmatched_iso_codes = pd.DataFrame(columns=['iso3','state_name','CCode'])
unmatched_iso_codes['state_name'] = unmatched_alliance_entries['StateNme']
unmatched_iso_codes['CCode'] = unmatched_alliance_entries['CCode']

# Find all the common iso codes for unmatched entries
for i in trade_iso_codes.iterrows():
    state_name = i[1]['state_name']
    if state_name in unmatched_iso_codes['state_name'].values:
        # Find index of row with state_name
        row_index = unmatched_iso_codes.loc[unmatched_iso_codes['state_name'] == state_name].index[0]
        # Replace NaN with iso3 code
        unmatched_iso_codes.loc[row_index,'iso3'] = i[1]['iso3']

# The 44 states with no iso codes, added where applicable manually
overwrite = False # Manually change if file overwrite needed
incomplete_manual_path = './data/manual_incomplete.csv'
# Check if file already written
if not(os.path.exists(incomplete_manual_path)) or overwrite:
    manual_iso_codes = unmatched_iso_codes[unmatched_iso_codes['iso3'].isna()]
    manual_iso_codes.to_csv(incomplete_manual_path, index=False)

# I manually added iso codes
complete_manual_path = './data/manual_complete.csv'
manual_iso_codes = pd.read_csv(complete_manual_path)

# Remove the duplicate iso codes from the new dataframe
for i in manual_iso_codes.iterrows():
    iso_code = i[1]['iso3']
    if iso_code in matched_entries['iso3'].values:
        # Remove the entry from unmatched_entries
        manual_iso_codes = manual_iso_codes[~(manual_iso_codes['iso3'] == iso_code)]

# Merge the dataframes and write to csv if not already written
overwrite = False # Manually change if file overwrite needed
common_iso_path = './data/common_iso_codes.csv'
# Check if file already written
if not(os.path.exists(common_iso_path)) or overwrite:
    final_iso_codes = pd.merge(matched_entries.drop(['StateAbb'],axis=1), manual_iso_codes, on=['iso3','state_name','CCode'], how='outer')
    final_iso_codes.to_csv(common_iso_path, index=False)

In [6]:
def find_trade_value(reporter,partner,years_list,session):
    """
    Sends API request to WITS for export and import data of reporter to partner for the given years.

    :param reporter: ISO code of reporter country
    :param partner: ISO code of partner country
    :param years_list: year(s) for the data as a list of integers or 'ALL' for all years
    :param session: requests session object for connection pooling
    :return: list of dictionaries representing trade value
    """

    if years_list == 'ALL':
        years_str = 'ALL'
    else:
        years_str = ';'.join(years_list)

    for data_type_index in range(2):
        if data_type_index == 0:
            # Export on first run
            api_indicator = 'XPRT-TRD-VL'
        else:
            # Import on second run
            api_indicator = 'MPRT-TRD-VL'

        url = f"http://wits.worldbank.org/API/V1/SDMX/V21/datasource/tradestats-trade/reporter/{reporter}/year/{years_str}/partner/{partner}/product/total/indicator/{api_indicator}"

        response = session.get(url)

        if response.status_code == 200:
            # Parse the XML response
            root = ET.fromstring(response.content)

            # Initialize lists to store data
            if data_type_index == 0:
                data_dict = {}

            series = root.find(".//Series")

            if series is not None:
                # Extract series key values
                reporter = series.get("REPORTER")
                partner = series.get("PARTNER")
                indicator = series.get("INDICATOR")

                for obs in series.findall(".//Obs"):
                    time_period = obs.get("TIME_PERIOD")
                    obs_value = obs.get("OBS_VALUE")
                    if indicator == 'XPRT-TRD-VL':
                        data_dict[time_period] = {'reporter':reporter, 'partner':partner, 'year':time_period, 'export':obs_value,'import':None}
                    else:
                        if time_period in data_dict:
                            data_dict[time_period]['import'] = obs_value
                        else:
                            data_dict[time_period] = {'reporter':reporter, 'partner':partner, 'year':time_period, 'export':None,'import':obs_value}
                if data_type_index == 1:
                    # Convert dictionary to list of dictionaries
                    return [inner_dict for inner_dict in data_dict.values()]
            else:
                print(f"No series found in the response for {api_indicator}.")
                if data_type_index == 1:
                    return None
        else:
            print(f"Failed to retrieve data for {reporter} {partner} {years_list} {api_indicator}. Status code:", response.status_code)
            return None

#session = requests.Session()
#pd.DataFrame(find_trade_value('DZA','AFG','ALL',session))
#float(entry_s['ObsValue'])*1000

In [7]:
# Function to find all trade data for a specific country
def trade_country_to_csv(iso_reporter,trade_iso_df,session):
    """
    Finds all trade data for a specific country and writes it to a csv file.
    :param iso_reporter: iso code for the specific country.
    :param trade_iso_df: DataFrame containing iso codes for all countries.
    :param session: requests session object for connection pooling.
    """
    trade_data_list = []
    for iso_partner in trade_iso_df['iso3']:
        #if iso_reporter != iso_partner: This comparison probably not efficient

        # API request for trade data
        trade_data = find_trade_value(iso_reporter,iso_partner,'ALL',session)
        if trade_data is not None:
            trade_data_list.extend(trade_data)

    trade_data_df = pd.DataFrame(trade_data_list)
    trade_data_df.to_csv(f'./trade_cache/trade_data_{iso_reporter}.csv', index=False)

In [8]:
# Get trade data for all countries
overwrite = False # Manually change if file overwrite needed
if overwrite:
    # Initialise list and dataframe to store data
    trade_iso_path = "./data/trade_country_iso.csv"
    trade_iso_df = pd.read_csv(trade_iso_path,encoding='latin1')
    no_of_countries = trade_iso_df.shape[0]

    # Create a session to take advantage of connection pooling for efficiency
    session = requests.Session()

    # Iterate through all combinations of countries with its own thread
    with concurrent.futures.ThreadPoolExecutor(max_workers=no_of_countries) as executor:
            # Submit tasks for each country combination
            future_to_thread_number = {
                executor.submit(trade_country_to_csv, trade_iso_df['iso3'][thread_number], trade_iso_df, session): thread_number
                for thread_number in range(no_of_countries)
            }
print("All trade data saved.")

All trade data saved.


In [9]:
overwrite = False # Manually change if file overwrite needed
trade_path = "./data/trade_data.csv"

# Check if data concatenating already done
if not(os.path.exists(trade_path)) or overwrite:
    # Combining all trade data into one dataframe and writing to a csv file
    trade_iso_path = "./data/trade_country_iso.csv"
    trade_iso_df = pd.read_csv(trade_iso_path,encoding='latin1')
    trade_data = pd.DataFrame(columns=['reporter','partner','year','export','import'])
    empty_isos = []
    dataframes = []

    for iso_code in trade_iso_df['iso3']:
        try:
            country_df = pd.read_csv(f'./trade_cache/trade_data_{iso_code}.csv')
            dataframes.append(country_df)
        except EmptyDataError:
            empty_isos.append(iso_code)

    trade_data = pd.concat(dataframes)
    trade_data.to_csv(trade_path, index=False)

    empty_list = []
    for iso in empty_isos:
        state_name = trade_iso_df[trade_iso_df['iso3'] == iso]['state_name'].values[0]
        empty_list.append({'iso3':iso,'state_name':state_name})

    empty_df = pd.DataFrame(empty_list)

    empty_path = './data/empty_countries.csv'
    empty_df.to_csv(empty_path,index=False)

In [5]:
"""Meant to be getting country meta data which it does but it's not very useful and doesn't
say which countries belong to which group"""
run = False
if run:
    url = 'http://wits.worldbank.org/API/V1/wits/datasource/tradestats-trade/country/ALL'
    response = requests.get(url)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        print(root.keys())

ParseError: syntax error: line 3, column 10 (<string>)